In [54]:
!pip install pandas

In [55]:
import pandas as pd

# Load the TSV file into a DataFrame
file_path = './supplier_questions.tsv' # change to your questions file
df = pd.read_csv(file_path, sep='\t')

# Check the content of the DataFrame (just to ensure it's loaded correctly)
df.head()

# concat the questions in the DataFrame
concatenated_questions = "\n".join(str(question) for question in df['Question'])
print("concatenated_questions:\n" + concatenated_questions)


concatenated_questions:
Is the supplier ISO 27001 certified?
Is the supplier SOC2 certified?


In [56]:
# Load the TSV file into a DataFrame
file_path = './context.tsv' # change to your resource file
df = pd.read_csv(file_path, sep='\t')

# Check the content of the DataFrame (just to ensure it's loaded correctly)
df.head()

# concat the questions in the DataFrame
concatenated_resources = "\n".join(str(res) for res in df['Resources'])
print("concatenated_resources:\n" + concatenated_resources)

concatenated_resources:
https://sentry.io/legal/dpa/5.1.0/in-app/?userCurrentVersion=5.1.0
https://sentry.io/security/
https://sentry.io/legal/terms/3.0.0/in-app/?userCurrentVersion=3.0.1


In [57]:
base_prompt = """
Below is a list of questions and resources.
Please answer the questions based on the resources provided:
"""

In [58]:
prompt = base_prompt + "\nQuestions:\n" + concatenated_questions + "\n\nResources:\n" + concatenated_resources
print("prompt:\n" + prompt)

prompt:

Below is a list of questions and resources.
Please answer the questions based on the resources provided:

Questions:
Is the supplier ISO 27001 certified?
Is the supplier SOC2 certified?

Resources:
https://sentry.io/legal/dpa/5.1.0/in-app/?userCurrentVersion=5.1.0
https://sentry.io/security/
https://sentry.io/legal/terms/3.0.0/in-app/?userCurrentVersion=3.0.1


In [59]:
anthropic_system = """
You're an ISO leading a compliance team seeking information about a potential future supplier. 
Your team is asked to perform a compliance due diligence. 
Respond in short and clear explanations and including URL resources if available.
Don't be lazy!
"""

In [60]:
!pip install anthropic
!pip install python-dotenv

In [61]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('ANTHROPIC_API_KEY')
if api_key is None:
    raise ValueError("The 'ANTHROPIC_API_KEY' environment variable is not set.")

In [62]:
import anthropic
client = anthropic.Anthropic()

def ask(prompt):
    response = client.messages.create(
        model="claude-3-5-sonnet-latest",
        max_tokens=2000,
        temperature=0.0,
        system=anthropic_system,
        messages=[{"role": "user", "content": prompt}]
    )
    print ("response completion: " + response['completion'])
    print ("response content text: " + response.content[0].text)
    return response['completion']

In [67]:
import json

min_confidence = 1.0
json_results = []
backfeed = ""
iteration_count = 0

while min_confidence > 0.3 and iteration_count < 10:
    iteration_count += 1

    evolving_prompt = prompt + "\n\n" + backfeed

    try:
        res = ask(evolving_prompt)
    except Exception as e:
        print(f"Exception occurred: {e.message}")
        print(f"Exception type: {type(e)}")
        break

    json_result = json.loads(res)

    if len(json_result) == 0:
        break

    min_confidence = min([i['confidence'] for i in json_result])
    # hier nochmal checken
    backfeed = "\n\nPlease try one more time and improve the confidence of your answer\n" + "\n".join([i['abbreviation'] for i in json_results])
    json_results.extend(json_result)
    print(str(json_results)[:100]) # print first 100 characters for debugging

Exception occurred: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}
Exception type: type(e)
